**Colab 的作業系統環境為不完整的 Debian ** 

可以勉強在 colab 環境上跑 Selenium，只是看不到，需要先在本機上細膩的操作，或也在本機操作一次 Selenium 環境，確認程序可用後，再移植到 Colab 上來。

In [17]:
#查詢
import platform
print(platform.platform())

#查詢目前 Google Chrome 的版本
!google-chrome-stable --version

Linux-5.10.147+-x86_64-with-glibc2.31
Google Chrome 111.0.5563.110 


In [2]:
!apt-get update
!apt-get install -y wget curl unzip gnupg
!wget -q -O - https://dl-ssl.google.com/linux/linux_signing_key.pub | apt-key add -
!echo "deb http://dl.google.com/linux/chrome/deb/ stable main" > /etc/apt/sources.list.d/google-chrome.list
!apt-get update
!apt-get install -y google-chrome-stable
#需查看一下 Chromedriver 能用的版本：https://chromedriver.storage.googleapis.com/
!wget -N https://chromedriver.storage.googleapis.com/111.0.5563.64/chromedriver_linux64.zip
!unzip chromedriver_linux64.zip
!chmod +x chromedriver


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 https://dl.google.com/linux/chrome/deb stable InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:6 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.68

In [3]:
#查詢 Chromedriver 的路徑，通常是 /usr/bin/chromedriver
!which chromedriver

#設定 os 環境路徑
import os
os.environ["PATH"] += os.pathsep + "/usr/bin/chromedriver"

In [5]:
# 安裝 Selenium
!pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 KB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 4.8 MB/s eta 0:00:00


In [6]:

# set options to be headless, ..
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By

options = webdriver.ChromeOptions()
#找 user-agent 的網站： https://www.whatsmyua.info/
userAgent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36";
#許多網站會檢查 user-agent
options.add_argument("user-agent={}".format(userAgent))
#不讓瀏覽器執行在前台，而是在背景執行，開發前期(或在本機執行)先不加才看的到畫面
options.add_argument('--headless')
# 在非沙盒測試環境下，可以 root 權限運行
options.add_argument('--no-sandbox')
# 不以 /dev/shm 作為暫存區(使用 /tmp)
options.add_argument('--disable-dev-shm-usage')
# 設定瀏覽器的解析度
# options.add_argument('--window-size=1920,1080')
# 關閉 GPU ，Google 文件提到需要加上這個參數以解決部份的 bug
# options.add_argument('--disable-gpu')

# open it, go to a website, and get results
wd = webdriver.Chrome(options=options)

需要使用 Selenium 進行爬蟲的網站：


1.   該網站有多道防爬蟲機制，因此使用 Selenium 來爬可以降低被擋的機會
2.   該網站的內容加載不是用換頁的，而是類似延遲加載的方式 Lazy Loading 
3.   該網站是使用前端語言框架所寫的 Single Page App (SPA)架構，內容更動都不會變動網址。
4.   需要頻繁且複雜的互動操作才能擷取到內容的網站，例：有會員機制或彈出視窗阻擋操作等機制。
5.   自動訂票、限時搶購的網站操作。
6.   需要經常填寫資料(例：生產履歷)，但該網站卻沒提供 API



In [11]:
#此網址是 Nasdaq 網站的 Tesla 即時股價，此股價在換頁時，網址並不會更動，因此適合使用 Selenium
baseUrl = 'https://www.nasdaq.com/market-activity/stocks/tsla/latest-real-time-trades'
wd.get(baseUrl)
#隱式等待：是在嘗試發現某個元素的時候，如果沒能立刻發現，就等待固定長度的時間。預設設定是0秒。
wd.implicitly_wait(5)

In [ ]:
# 傾印網頁內容，以測試網頁是否有正確(權限)讀取完整內容
# 注意：內容會很長
# print(wd.page_source) 

In [12]:
results = wd.find_element(By.XPATH, '//table[@class="latest-real-time-trades__table"]').get_attribute('innerHTML')
print(results)


          <thead class="latest-real-time-trades__table-headings">
            <tr class="latest-real-time-trades__row latest-real-time-trades__row--headings"><th class="latest-real-time-trades__table-heading" scope="col">NLS Time (ET)</th><th class="latest-real-time-trades__table-heading" scope="col">NLS Price</th><th class="latest-real-time-trades__table-heading" scope="col">NLS Share Volume</th></tr>
          </thead>
          <tbody class="latest-real-time-trades__table-body"><tr class="latest-real-time-trades__row"><th scope="row" class="latest-real-time-trades__cell">16:00:04</th><td scope="row" class="latest-real-time-trades__cell">$ 190.41</td><td scope="row" class="latest-real-time-trades__cell">300</td></tr><tr class="latest-real-time-trades__row"><th scope="row" class="latest-real-time-trades__cell">16:00:04</th><td scope="row" class="latest-real-time-trades__cell">$ 190.41</td><td scope="row" class="latest-real-time-trades__cell">10,500</td></tr><tr class="latest-real-tim

In [16]:
# 向下捲動捲軸 500 像素
wd.execute_script("window.scrollBy(0, 500)")
wd.implicitly_wait(5)
#點擊下一頁
nextButton = wd.find_element(By.XPATH, '//button[@class="pagination__next"]')
nextButton.click()
# 顯示等待：等待目標元素的出現，最多等待10秒
element = WebDriverWait(wd, 10).until(expected_conditions.presence_of_element_located((By.XPATH, '//div[@class="latest-real-time-trades__data"]')))

results = wd.find_element(By.XPATH, '//table[@class="latest-real-time-trades__table"]').get_attribute('innerHTML')
print(results)


          <thead class="latest-real-time-trades__table-headings">
            <tr class="latest-real-time-trades__row latest-real-time-trades__row--headings"><th class="latest-real-time-trades__table-heading" scope="col">NLS Time (ET)</th><th class="latest-real-time-trades__table-heading" scope="col">NLS Price</th><th class="latest-real-time-trades__table-heading" scope="col">NLS Share Volume</th></tr>
          </thead>
          <tbody class="latest-real-time-trades__table-body"><tr class="latest-real-time-trades__row"><th scope="row" class="latest-real-time-trades__cell">16:00:04</th><td scope="row" class="latest-real-time-trades__cell">$ 190.41</td><td scope="row" class="latest-real-time-trades__cell">172</td></tr><tr class="latest-real-time-trades__row"><th scope="row" class="latest-real-time-trades__cell">16:00:04</th><td scope="row" class="latest-real-time-trades__cell">$ 190.41</td><td scope="row" class="latest-real-time-trades__cell">100</td></tr><tr class="latest-real-time-t